# Introducing Checklist Component

In [1]:
import pandas as pd

import plotly.express as px
from dash import Dash, html, dcc, Input, Output

## 1. Getting and exploring the data

In [2]:
# Load the dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00616/Tetuan%20City%20power%20consumption.csv')

In [3]:
# Changing DateTime column to datetime
df.DateTime = pd.to_datetime(df.DateTime)

In [4]:
# Renaming some columns
df.rename(columns={'Zone 1 Power Consumption': 'Zone1', 'Zone 2  Power Consumption': 'Zone2', 
                   'Zone 3  Power Consumption': 'Zone3'}, inplace=True)

In [5]:
# Getting one value per day
dfd = df.groupby(pd.Grouper(key='DateTime', freq='1D')).mean().reset_index()
dfd.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone1,Zone2,Zone3
0,2017-01-01,9.675299,68.519306,0.315146,121.390771,25.993924,28465.232067,17737.791287,17868.795181
1,2017-01-02,12.476875,71.456319,0.076563,120.404486,27.227410,28869.493671,19557.725431,17820.763053
2,2017-01-03,12.100000,74.981667,0.076715,120.686014,28.574660,30562.447257,20057.269504,17620.803213
3,2017-01-04,10.509479,75.459792,0.082417,122.959319,28.827222,30689.831224,20102.077001,17673.694779
4,2017-01-05,10.866444,71.040486,0.083896,118.749861,29.741437,30802.911393,20033.941237,17664.176707


In [6]:
zones = ['Zone1','Zone2','Zone3']

In [7]:
line_fig = px.line(dfd, x='DateTime', y=zones, 
                    hover_data={'value':':.2f'},
                    title=f'Power Consumption')
line_fig.update_yaxes(range=[0, dfd.Zone1.max()+5000])
line_fig.show()

## 2. Building the layout 

In [8]:
# Creating the Dash app
app = Dash()

In [9]:
app.layout = html.Div(children=[
    # Defining the title
    html.H1(id='H1', children='Power Consumption in Tetouan, 2017', 
            style={'textAlign':'center', 'color':'red'}),
    # Defining a subtitle              
    html.H3(id='H3', children='(Introducing the Checklist Component)', 
            style={'textAlign':'center'}),      
    # Defining the Checklist component 
    dcc.Checklist(id='zone_check',
        options={
            'Zone1': 'Zone 1',
            'Zone2': 'Zone 2',
            'Zone3': 'Zone 3',                
        },
        value=['Zone1', 'Zone2', 'Zone3'],
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ),                   
    # Defining the graph with id='line-gr'
    dcc.Graph(id='line-gr')
], style={'backgroundColor':'white'})

## 3. Adding interactivity with callback functions

This dashboard will have one `Input`, the `RadioItems` value, and one `Output`, a graph.

In [10]:
# Set up the callback function
@app.callback(
    Output('line-gr',    'figure'),
    Input ('zone_check', 'value')
)
def update_graph(selected_zones):
    # getting the zones to plot    
    zones = [z for z in ['Zone1','Zone2','Zone3'] if z in selected_zones]

    line_fig = px.line(dfd, x='DateTime', y=zones, 
                    hover_data={'value':':.2f'},
                    title='Power Consumption')
    line_fig.update_yaxes(range=[0, dfd.Zone1.max()+5000])  
    return line_fig

## 4. Running the dashboard

In [11]:
# Run app 
if __name__ == '__main__':
    app.run_server(debug=True, port=8090)

The dashboard looks like this.

<img src="checkDash.gif" align="center">

## References

- https://dash.plotly.com/dash-core-components/checklist
- Salam, A., & El Hibaoui, A. (2018) "Comparison of Machine Learning Algorithms for the Power Consumption Prediction: -Case Study of Tetouan city". In 2018 6th International Renewable and Sustainable Energy Conference (IRSEC) (pp. 1-5)